#### package

In [1]:
from typing import Optional, Callable, List, Tuple
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
import torch
import torch.nn as nn
import torch.nn.functional as F
from mamba import Mamba, MambaConfig
from loss_functions import get_loss_fn
from datetime import datetime
from sklearn.decomposition import PCA
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from scipy.stats import pearsonr
import random
from sklearn.preprocessing import StandardScaler
from scipy.stats import spearmanr
from sklearn.preprocessing import PowerTransformer
from dataclasses import dataclass
import os
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import TensorDataset, DataLoader
import gc

/home/luchinwei/anaconda3/envs/mambastock/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### hyperparameters

In [2]:
save_dir = '0730_2330/'
use_cuda = True
n_steps=10
window=30
patience = 100
val_ratio = 0.2
epochs = 1000
loss_fcn = "mse"

lr = 0.0005
wd = 1e-5
hidden = 32
layer = 3
n_test = 350
ts_code = 2330
risk_free = 0.017

In [3]:
def generate_return_features(data, price_col='close', windows=[5, 10, 20]):
    import pandas as pd

    df = data.copy()
    returns = df[price_col].pct_change()

    # 基本報酬率
    df['returns'] = returns

    for window in windows:
        # 移動平均報酬率
        df[f'returns_ma_{window}'] = returns.rolling(window=window).mean()
        
        # 指數移動平均
        df[f'returns_ema_{window}'] = returns.ewm(span=window, adjust=False).mean()

        # 報酬率波動度
        df[f'returns_volatility_{window}'] = returns.rolling(window=window).std()

        # 報酬率的 Z-score
        mean = returns.rolling(window=window).mean()
        std = returns.rolling(window=window).std()
        df[f'returns_zscore_{window}'] = (returns - mean) / std

    return df.dropna()


#### Import Data

In [4]:
data = pd.read_csv(str(ts_code)+"_value"+'.csv')
data['trade_date'] = pd.to_datetime(data['trade_date'], format='%Y/%m/%d')

# 加入星期幾（0=星期一, 6=星期日）
data['day_of_week'] = data['trade_date'].dt.dayofweek

# 加入月份（1～12）
data['month'] = data['trade_date'].dt.month

returns = data['close_TW_roc'] 

tp=[5, 10, 20]

for tp in tp:
    # 移動平均報酬率
    data[f'returns_ma_{tp}'] = returns.rolling(window=tp).mean()
    
    # 指數移動平均
    data[f'returns_ema_{tp}'] = returns.ewm(span=tp, adjust=False).mean()

    # 報酬率波動度
    data[f'returns_volatility_{tp}'] = returns.rolling(window=tp).std()

    # 報酬率的 Z-score
    mean = returns.rolling(window=tp).mean()
    std = returns.rolling(window=tp).std()
    data[f'returns_zscore_{tp}'] = (returns - mean) / std

data = data.dropna().reset_index(drop=True)

'''
['trade_date', 'stock_no', 'stock_name', 'stock_type', 
'open_TW', 'close_TW', close_TW_roc, 'h_TW', 'l_TW', 'vol_TW', 'CBOE_SKEW_INDEX', 
'CBOE_Volatility_INDEX', 'COPPER_F', 'COPPER', 'GOLD_F', 
'GOLD', 'OIL_F', 'OIL', 'SP_F', 'SILVER_F', 'SILVER', 
'TWII', 'IXIC', 'GSPC', 'DJI', 'NYSE', 'RUSSELL', 'SSE', 
'FCHI', 'FTSE', 'GDAXI', 'Nikkei_F', 'IXIC_F', 'DJI_F', 
'S_P_F', 'USDX_F', 'JPY', 'GBP', 'HKD', 'CNY', 'AUD', 
'TWD', 'EUR', 'DTB4WK', 'DTB3', 'DTB6', 'DGS5', 'DGS1', 
'DAAA', 'DBAA', 'DGS3MO', 'DGS6MO', 'DGS1.1', 'DCOILWTICO', 
'TE1', 'TE2', 'TE3', 'TE5', 'TE6', 'DE1', 'DE2', 'DE4', 'DE5', 
'DE6', 'DCOILWTICO_rel_change', 'day_of_week', 'month']
'''

index = ['open_TW', 'close_TW', 'h_TW', 'l_TW', 'vol_TW', 'CBOE_SKEW_INDEX', 
'CBOE_Volatility_INDEX', 'TWII', 'DJI', 'NYSE', 'TWD', 'CNY', 'day_of_week', 'month',
"returns_ma_5", "returns_ema_5", "returns_volatility_5","returns_zscore_5",
"returns_ma_10", "returns_ema_10", "returns_volatility_10", "returns_zscore_10",
"returns_ma_20", "returns_ema_20", "returns_volatility_20", "returns_zscore_20"]

# 拆離 label 欄位

ratechg = data['close_TW_roc'].values

features = data[index].values
num_samples = features.shape[0]
# num_samples: 2520

# 定義 horizon 數
horizon = n_steps

X_list, y_list = [], []
for i in range(num_samples - window - horizon + 1):
    X_win = features[i : i + window]                        # shape: [window, features]
    t = np.arange(window).reshape(-1, 1)  # shape: [window, 1]
    sin_t = np.sin(t / 10000)
    cos_t = np.cos(t / 10000)
    time_encoding = np.concatenate([sin_t, cos_t], axis=1)    # shape: [window, 2]

    X_win = np.concatenate([X_win, time_encoding], axis=1) 
    y_future = ratechg[i + window : i + window + horizon]     # shape: [horizon]
    X_list.append(X_win)
    y_list.append(y_future)

trainX = np.array(X_list[:-n_test])
trainy = np.array(y_list[:-n_test])
testX = np.array(X_list[-n_test:])
testy = np.array(y_list[-n_test:])

# trainX.shape: (2146, 15, 61)  2146 = num_samples - window - horizon + 1
# trainy.shape: (2146, 10)
# testX.shape: (350, 15, 61)
# testy.shape: (350, 10)

#### train / val

In [5]:
val_size = int(len(trainX) * val_ratio)

valX = trainX[-val_size:]
valy = trainy[-val_size:]

trainX = trainX[:-val_size]
trainy = trainy[:-val_size]

# trainX.shape: (1717, 15, 61)
# trainy.shape: (1717, 10)
# valX.shape: (429, 15, 61)
# valy.shape: (429, 10)

in_dim = trainX.shape[2]

#### DataLoader

In [6]:
trainX_tensor = torch.tensor(trainX, dtype=torch.float32)
trainy_tensor = torch.tensor(trainy, dtype=torch.float32)

valX_tensor = torch.tensor(valX, dtype=torch.float32)
valy_tensor = torch.tensor(valy, dtype=torch.float32)

testX_tensor = torch.tensor(testX, dtype=torch.float32)
testy_tensor = torch.tensor(testy, dtype=torch.float32)

# Build datasets and loaders
train_dataset = TensorDataset(trainX_tensor, trainy_tensor)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

val_dataset = TensorDataset(valX_tensor, valy_tensor)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

test_dataset = TensorDataset(testX_tensor, testy_tensor)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

#### 模型

In [7]:
class Net(nn.Module):
    """
    多層 Mamba 時序模型，用於序列輸入特徵的未來預測。

    參數:
        in_dim (int): features 數量
        horizon (int): 模型預測的未來時間步數（輸出維度）。
        hidden (int): 中間表示層（latent layer）的維度。
        layer (int): Mamba block 的堆疊層數。

    屬性:
        config (MambaConfig): 用於初始化 Mamba 模型的配置。
        proj_in (nn.Linear): 將輸入特徵從 in_dim 投影到 hidden 維度。
        mamba (Mamba): 多層 Mamba 模型，用於處理時間序列資料。
        pool (nn.AdaptiveAvgPool1d): 算平均
        head (nn.Linear): 轉換為 horizon 維度的預測向量。
    """
    in_dim: int
    horizon: int
    hidden: int
    layer: int

    config: MambaConfig
    proj_in: nn.Linear
    mamba: Mamba
    pool: nn.AdaptiveAvgPool1d
    head: nn.Linear

    def __init__(self, in_dim: int, horizon: int, hidden: int, layer: int) -> None:
        super().__init__()
        self.in_dim = in_dim
        self.horizon = horizon
        self.hidden = hidden
        self.layer = layer

        self.config = MambaConfig(d_model=hidden, n_layers=layer)
        self.proj_in = nn.Linear(in_dim, hidden)
        self.mamba = Mamba(self.config)
        self.pool = nn.AdaptiveAvgPool1d(1)
        self.head = nn.Linear(hidden, horizon)

    def forward(self, x: torch.Tensor) -> torch.Tensor: 
        x = self.proj_in(x)            # x: [B, window, in_dim] → [B, window, hidden]
        x = self.mamba(x)              #                        → [B, window, hidden]
        x = x.transpose(1, 2)          #                        → [B, hidden, window]
        x = self.pool(x).squeeze(-1)   #                        → [B, hidden]
        return self.head(x)            #                        → [B, horizon]

In [8]:
class MLPNet(nn.Module):
    """
    Fully Connected MLP 模型，用於序列特徵的未來預測。

    每個時間步的特徵會被展平成一個長向量，送進 MLP 層中做預測。

    輸入:
        x: [B, window, in_dim] → 展平 → [B, window * in_dim]

    輸出:
        預測向量 [B, horizon]
    """
    in_dim: int
    horizon: int
    hidden: int
    window: int
    
    def __init__(self, in_dim: int, horizon: int, hidden: int, window: int) -> None:
        super().__init__()
        self.flatten_dim = window * in_dim
        self.fc = nn.Sequential(
            nn.Linear(self.flatten_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, hidden),
            nn.ReLU(),
            nn.Linear(hidden, horizon)
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = x.view(x.size(0), -1)  # [B, window * in_dim]
        return self.fc(x)          # → [B, horizon]

#### dataclass

In [9]:
@dataclass
class DataBatch:
    train_loader: DataLoader
    val_loader: DataLoader
    test_loader: DataLoader

#### Metrics

In [10]:
def compute_pearson(pred, target):
    pred_np = pred.detach().cpu().view(-1).numpy().astype(np.float64)
    target_np = target.detach().cpu().view(-1).numpy().astype(np.float64)
    return pearsonr(pred_np, target_np)[0]


#### Trainer

In [11]:
class Trainer:

    device: str 
    model: torch.nn.Module
    loss_fn: str
    opt: torch.optim.Optimizer
    train_losses: list
    val_losses: list
    
    
    def __init__(self, model: torch.nn.Module, loss_fn: str, opt: torch.optim.Optimizer, scheduler, device: str = "cuda"):
        self.model = model.to(device)
        self.loss_fn = get_loss_fn(loss_fn)
        self.opt = opt
        self.scheduler = scheduler
        self.device = device
        self.train_losses = []
        self.val_losses = []


    def fit(
        self,
        databatch: DataBatch,
        epochs: int,
        verbose: bool = True,
        filename: Optional[str] = None,
        patience: int = 10
    ) -> Tuple[List[float], List[float]]:

        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        counter = 0
        
        best_val_loss = float("inf")
        best_model_state = None
        best_epoch = -1
        
        for epoch in range(epochs):
            self.model.train()
            epoch_train_loss = 0.0

            for X_batch, y_batch in databatch.train_loader:
                X_batch = X_batch.to(self.device)
                y_batch = y_batch.to(self.device)

                pred = self.model(X_batch)
                loss = self.loss_fn(pred, y_batch)
                self.opt.zero_grad()
                loss.backward()
                self.opt.step()

                epoch_train_loss += loss.item()

            self.model.eval()
            with torch.no_grad():
                val_loss_total = 0.0
                for valX, valy in databatch.val_loader:
                    valX = valX.to(self.device)
                    valy = valy.to(self.device)
                    val_pred = self.model(valX)
                    val_loss_total += self.loss_fn(val_pred, valy).item()
                val_loss_avg = val_loss_total / len(databatch.val_loader)

            self.train_losses.append(epoch_train_loss / len(databatch.train_loader))
            self.val_losses.append(val_loss_avg)
            self.scheduler.step(val_loss_avg)
            

            current_lr = self.opt.param_groups[0]['lr']
            print(f"Epoch {epoch}: lr={current_lr:.6f}, ValLoss={val_loss_avg:.8f}")

            if val_loss_avg < best_val_loss:
                best_val_loss = val_loss_avg
                best_model_state = self.model.state_dict()
                best_epoch = epoch
                counter = 0
            else:
                counter += 1
                if counter >= patience:
                    print(f"Early stopping triggered at epoch {epoch+1}")
                    break

            if verbose and (epoch % 10 == 0 or epoch == epochs - 1):
                print(f"[Epoch {epoch+1}] Train: {epoch_train_loss / len(databatch.train_loader):.4f} | Val: {val_loss_avg:.4f}")
            
            gc.collect()
            torch.cuda.empty_cache()
            
        if filename:
            path = f"{filename}_{timestamp}.pth"
        else:
            path = f"checkpoint_{timestamp}.pth"       
        
        os.makedirs(os.path.dirname(path), exist_ok=True)
        
        torch.save({
        "model_state": best_model_state,
        "epoch": best_epoch,
        "val_loss": best_val_loss
        }, path)
        
        print(f"Best checkpoint saved at epoch {best_epoch+1}, Val Loss: {best_val_loss:.4f}")
        self.model.load_state_dict(best_model_state)
    
        preds, targets, test_loss, r2 = self.evaluate(databatch)
        pearson_corr = compute_pearson(preds, targets)

        log_entry = (
            f"{path}\n"
            f"R²: {r2:.4f} | Pearson: {pearson_corr:.4f}\n"
            f"{'-'*35}\n"
        )

        with open("training_log.txt", "a") as f:
            f.write(log_entry)
        
        return self.train_losses, self.val_losses


    def evaluate(
        self,
        databatch: DataBatch,
    ) -> Tuple[torch.Tensor, float]:

        self.model.eval()
        all_preds = []
        all_targets = []
        total_loss = 0.0

        with torch.no_grad():
            for testX, testy in databatch.test_loader:
                testX = testX.to(self.device)
                testy = testy.to(self.device)

                pred = self.model(testX)
                loss = self.loss_fn(pred, testy)

                all_preds.append(pred.cpu())
                all_targets.append(testy.cpu())
                total_loss += loss.item()

        preds = torch.cat(all_preds)
        targets = torch.cat(all_targets)
        r2 = r2_score(targets, preds)
        avg_loss = total_loss / len(databatch.test_loader)

        print(f"Test Loss: {avg_loss:.4f} | R² Score: {r2:.4f}")
        return preds, targets, avg_loss, r2

    
    
    def save_checkpoint(self, path: str):
        torch.save({
            "model_state": self.model.state_dict(),
            "optimizer_state": self.opt.state_dict(),
            "train_losses": self.train_losses,
            "val_losses": self.val_losses
        }, path)
        

    

#### 訓練

In [12]:
databatch = DataBatch(
    train_loader = train_loader,
    val_loader = val_loader,
    test_loader = test_loader,
)

regressor = Net(in_dim=in_dim, horizon=n_steps, hidden=hidden, layer=layer)
opt = torch.optim.Adam(regressor.parameters(), lr=lr, weight_decay=wd)
scheduler = ReduceLROnPlateau(opt, mode="min", factor=0.5, patience=5)
trainer = Trainer(model=regressor, loss_fn="smooth", opt=opt, scheduler=scheduler, device="cuda")
trainer.fit(databatch=databatch, epochs=epochs, verbose=False, filename="0730_2330/2330", patience=10)

Epoch 0: lr=0.000500, ValLoss=0.00016115
Epoch 1: lr=0.000500, ValLoss=0.00015892
Epoch 2: lr=0.000500, ValLoss=0.00016789
Epoch 3: lr=0.000500, ValLoss=0.00016255
Epoch 4: lr=0.000500, ValLoss=0.00016028
Epoch 5: lr=0.000500, ValLoss=0.00016149
Epoch 6: lr=0.000500, ValLoss=0.00016686
Epoch 7: lr=0.000250, ValLoss=0.00016275
Epoch 8: lr=0.000250, ValLoss=0.00016007
Epoch 9: lr=0.000250, ValLoss=0.00016205
Epoch 10: lr=0.000250, ValLoss=0.00015893
Epoch 11: lr=0.000250, ValLoss=0.00016261
Early stopping triggered at epoch 12
Best checkpoint saved at epoch 2, Val Loss: 0.0002
Test Loss: 0.0003 | R² Score: -0.0152


([0.004053785001849734,
  0.00012662085790444112,
  0.0001266457106537001,
  0.0001264566976801088,
  0.00012565660956238817,
  0.00012647017666615352,
  0.00012703021227180043,
  0.000127888334182504,
  0.0001235702438862063,
  0.00012478622313279148,
  0.0001248644220363788,
  0.00012476851688401403],
 [0.00016114591906241712,
  0.00015891716564645457,
  0.00016788772936032965,
  0.00016254696169602944,
  0.00016028373486026086,
  0.00016148765327536103,
  0.00016686107078890523,
  0.00016274588812882062,
  0.00016007227269357018,
  0.00016205029634066805,
  0.00015893236101234193,
  0.00016261404892214127])